# SqlAlchemy Core


SqlAlchemy Core는 SqlAlchemy의 ORM의 핵심 철학인 객체 기반이 아니라 정통 SQL 방식의 기반처리를 담당하는 부분이며 
SqlAlchemy ORM은 SqlAlchemy Core를 내부에서 사용한다.

SqlAlchemy Core는 다음과 같은 특징을 가진다.

- 테이블별로 별도의 클래스를 사용하지 않고 모든 테이블을 `Table`이라는 하나의 공통 클래스로 처리
- SQL 표현언어(Expression Language)에 기반한 SQL 1대1 연동 문법


## 테이블 정의

SqlAlchemy Core에서 각각의 테이블은 `Table` 클래스 객체로 생성되며 이렇게 생성된 `Table` 클래스 객체는 `MetaData` 클래스 객체에서 통합 관리한다.

`Table` 클래스 객체를 생성하기 위해서는 인수로 `MetaData` 클래스 객체가 필요하므로 우선 다음과 같이 `MetaData` 클래스 객체를 생성해야 한다.

In [1]:
import sqlalchemy as sa

metadata_obj = sa.MetaData()

metadata_obj

MetaData()

`Table` 객체 생성자는 실제 데이터베이스의 테이블 이름 문자열과 MetaData 객체, 그리고 각 열을 정의하는 `Column` 객체를 인수로 받는다.

```python
Table_객체 = Table(
  "실제_데이터베이스의_테이블_이름_문자열",
  MetaData_객체,
  Column_객체,
  Column_객체,
  Column_객체,
  ...
)
```

`Column` 객체 생성자는 ORM에서 사용하는 열 이름 문자열과, 열 데이터 타입, 그리고 기타 설정사항등을 인수로 받는다.

```python
Column_객체 = Column(
  "ORM에서_사용하는_열_이름_문자열",
  열_데이터_타입,
  ...(기타 설정사항)...
)
```

많이 사용되는 기타 설정사항 인수으로는 `key`, `primary_key`, `nullable` 등이 있다.

- `key`: 데이터베이스 테이블에 생성되는 열의 실제 이름. ORM에서 사용하는 열 이름 문자열과 실제 열 이름을 다르게 하고 싶은 경우에 지정하며 이를 생략하면 첫번째 인수인 "ORM에서 사용하는 열 이름 문자열"을 그대로 사용한다.
- `primary_key`: 해당 열을 기본키로 지정하고 싶으면 `primary_key=True` 인수를 지정한다. 복합 기본키(compound primary key)를 사용하고 싶으면 각각의 열에 `primary_key` 인수를 지정하거나 `Table` 객체를 생성할 때 별도의 키 제한조건을 넣는다.
- `nullable`: Null 값 입력을 가능하게 하고 싶으면 `nullable=True` 인수를 지정한다. 디폴트 값은 `True`

또한 해당 열이 다른 테이블의 외래키인 경우 `ForeignKey("테이블_이름.열_이름")`의 형식으로 `ForeignKey` 객체를 넣는다.

다음은 `user`, `user_prefs` 두 개의 테이블을 정의하는 SqlAlchemy Core 예제코드다. 

In [2]:
user = sa.Table(
    "user",
    metadata_obj,
    sa.Column("user_id", sa.Integer, primary_key=True),
    sa.Column("user_name", sa.String(16), nullable=False),
    sa.Column("email_address", sa.String(60), key="email"),
    sa.Column("nickname", sa.String(50), nullable=False),
)

user

Table('user', MetaData(), Column('user_id', Integer(), table=<user>, primary_key=True, nullable=False), Column('user_name', String(length=16), table=<user>, nullable=False), Column('email_address', String(length=60), table=<user>, key='email'), Column('nickname', String(length=50), table=<user>, nullable=False), schema=None)

In [3]:
user_prefs = sa.Table(
    "user_prefs",
    metadata_obj,
    sa.Column("pref_id", sa.Integer, primary_key=True),
    sa.Column("user_id", sa.Integer, sa.ForeignKey("user.user_id"), nullable=False),
    sa.Column("pref_name", sa.String(40), nullable=False),
    sa.Column("pref_value", sa.String(100)),
)

user_prefs

Table('user_prefs', MetaData(), Column('pref_id', Integer(), table=<user_prefs>, primary_key=True, nullable=False), Column('user_id', Integer(), ForeignKey('user.user_id'), table=<user_prefs>, nullable=False), Column('pref_name', String(length=40), table=<user_prefs>, nullable=False), Column('pref_value', String(length=100), table=<user_prefs>), schema=None)

생성된 테이블 객체 정보는 MetaData 객체의 `tables` 및 `sorted_tables` 속성에 각각 딕셔너리 타입 및 리스트 타입 값으로 포함된다.

In [4]:
metadata_obj.tables

FacadeDict({'user': Table('user', MetaData(), Column('user_id', Integer(), table=<user>, primary_key=True, nullable=False), Column('user_name', String(length=16), table=<user>, nullable=False), Column('email_address', String(length=60), table=<user>, key='email'), Column('nickname', String(length=50), table=<user>, nullable=False), schema=None), 'user_prefs': Table('user_prefs', MetaData(), Column('pref_id', Integer(), table=<user_prefs>, primary_key=True, nullable=False), Column('user_id', Integer(), ForeignKey('user.user_id'), table=<user_prefs>, nullable=False), Column('pref_name', String(length=40), table=<user_prefs>, nullable=False), Column('pref_value', String(length=100), table=<user_prefs>), schema=None)})

In [5]:
metadata_obj.sorted_tables

[Table('user', MetaData(), Column('user_id', Integer(), table=<user>, primary_key=True, nullable=False), Column('user_name', String(length=16), table=<user>, nullable=False), Column('email_address', String(length=60), table=<user>, key='email'), Column('nickname', String(length=50), table=<user>, nullable=False), schema=None),
 Table('user_prefs', MetaData(), Column('pref_id', Integer(), table=<user_prefs>, primary_key=True, nullable=False), Column('user_id', Integer(), ForeignKey('user.user_id'), table=<user_prefs>, nullable=False), Column('pref_name', String(length=40), table=<user_prefs>, nullable=False), Column('pref_value', String(length=100), table=<user_prefs>), schema=None)]

다음 코드로 `tables` 속성에 포함된 user 테이블 객체가 아까 정의한 user 테이블 객체임을 확인할 수 있다.

In [6]:
user is metadata_obj.tables["user"]

True

Table 객체의 `columns` 또는 `c` 속성은 해당 테이블의 Column 객체를 포함하며 다음과 같이 접근할 수 있다.

In [7]:
user.columns.user_id

Column('user_id', Integer(), table=<user>, primary_key=True, nullable=False)

In [8]:
user.c.user_id

Column('user_id', Integer(), table=<user>, primary_key=True, nullable=False)

In [9]:
user.c["user_id"]

Column('user_id', Integer(), table=<user>, primary_key=True, nullable=False)

In [10]:
user.c.email

Column('email_address', String(length=60), table=<user>, key='email')

다음과 같이 복수의 Column 객체를 동시에 접근할 수도 있다.

In [11]:
user.c["user_id", "nickname"]

In [12]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:", echo=True)

In [13]:
# | output: false
%load_ext sql
%sql engine

Loading configurations from /Users/joelkim/.jupysql/config.

Settings changed:

Config,value
feedback,0
displaylimit,20
named_parameters,enabled


In [14]:
%sqlcmd tables

2024-02-15 16:04:57,970 INFO sqlalchemy.engine.Engine BEGIN (implicit; DBAPI should not BEGIN due to autocommit mode)
2024-02-15 16:04:57,971 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2024-02-15 16:04:57,972 INFO sqlalchemy.engine.Engine [raw sql] ()


Name


In [15]:
metadata_obj.create_all(engine)

2024-02-15 16:04:58,029 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-15 16:04:58,031 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user")
2024-02-15 16:04:58,032 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-15 16:04:58,034 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user")
2024-02-15 16:04:58,035 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-15 16:04:58,036 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_prefs")
2024-02-15 16:04:58,036 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-15 16:04:58,037 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_prefs")
2024-02-15 16:04:58,038 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-15 16:04:58,040 INFO sqlalchemy.engine.Engine 
CREATE TABLE user (
	user_id INTEGER NOT NULL, 
	user_name VARCHAR(16) NOT NULL, 
	email_address VARCHAR(60), 
	nickname VARCHAR(50) NOT NULL, 
	PRIMARY KEY (user_id)
)


2024-02-15 16:04:58,042 INFO sqlalchemy.engine.Engine [no key 0.00134s] ()
2024-02-

In [16]:
%sqlcmd tables

2024-02-15 16:04:58,057 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2024-02-15 16:04:58,060 INFO sqlalchemy.engine.Engine [raw sql] ()


Name
user
user_prefs
